In [1]:
import lacosmicx

In [2]:
import os
import mar
from astropy.io import fits
import numpy as np

from astropy.table import Table

from mar.config import MarManager

Could not import aqutil

Limited graphics available on OSX (aqutil not loaded)

Could not load pyraf!


In [3]:
from mar.reduction import PrepareSciImages

In [4]:
scim = PrepareSciImages(files = ["data/images/STRIPE82-20180812-075626.fits.fz"], outdir = "data/images")

In [5]:
masterBias = "data/masters/masterbias20180808-20180820.fits"
hot_mask = "data/masters/hotmask20180808-20180820.fits"

In [6]:
masterFlat = "data/masters/masterflat20180701-20180901.fits"
cold_mask = "data/masters/coldmask20180701-20180901.fits"

In [7]:
scim.outputfiles = False
scim.run_overscan(masterBias = masterBias, masterFlat = masterFlat)

1 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x166259d00>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x16624b0a0>]


In [8]:
file_path = "data/images/corr_STRIPE82-20180812-075626.fits"

In [9]:
hotmask = fits.open(hot_mask)
coldmask = fits.open(cold_mask)

In [10]:
## Opening image
file_data = mar.image.marfits.fromfile(
    file_path, 
    usemask = False
)

In [11]:
def create_starmask(file_pathh):
    sextr = mar.wrappers.SExtr(
        file_pathh,
        folder = "/Users/gustavo/MAR/dev/data/sextr/",
        catname = os.path.basename(file_pathh) + "_seg_.cat",
    )

    segmentation_image = f"data/sextr/{os.path.basename(file_pathh)}_seg_.fits"
    sextr.config["CHECKIMAGE_TYPE"]['value'] = "SEGMENTATION"
    sextr.config["CHECKIMAGE_NAME"]['value'] = segmentation_image
    sextr.params = [
        'NUMBER',
        'FLAGS'
    ]
    sextr.run()
    
    table = Table.read(os.path.join("data/sextr/", os.path.basename(file_pathh) + "_seg_.cat"), hdu=2)
    selection = (
        (table["FLAGS"] <= 2)
    )

    table = table[selection]
    segmentation_data = fits.getdata(segmentation_image)
    stars_mask = np.isin(segmentation_data, np.array(table["NUMBER"])).astype("int")
    fits.PrimaryHDU(data=stars_mask).writeto("data/tests/star_mask.fits", overwrite=True)
    return stars_mask

In [12]:
final_mask = mar.image.MaskArray(
        np.zeros(file_data[0].data.shape, dtype=np.uint8)
)
    

In [13]:
mask = hotmask[0].data + coldmask[0].data + (file_data[0].data >= int(mar.env.marConf['Instrument']['SATURATE']))
mask = mask > 0
mask = np.ma.masked_array(np.zeros(mask.shape), mask, fill_value=1)
mask = mask.filled(1).astype(np.uint8)

final_mask.add_mask(mask.astype(np.uint8), 1)

In [14]:
MarManager.INFO(f"applying first mask")
# important to not pass mask 64 here.  
file_data[0].data = lacosmicx.apply_mask(np.asarray(file_data[0].data, dtype=np.float32), mask)
file_data.writeto("data/tests/after_fm.fits", compress=False, overwrite=True)

In [15]:
star_mask = create_starmask("data/tests/after_fm.fits")
final_mask.add_mask(
    star_mask.astype(np.uint8), 64
)

In [16]:
fits.PrimaryHDU(data=
                final_mask.get_mask([1, 2, 64], binary=True).astype(np.uint8)
        ).writeto("data/tests/ignoremask.fits", overwrite=True)

In [17]:
MarManager.INFO(f"getting lacosmic")
cosmicMask, _ = mar.reduction.lacosmicx.lacosmicx(np.asarray(file_data[0].data, dtype=np.float32), final_mask.get_mask([1, 2, 64], binary=True).astype(np.uint8))
cosmicMask = np.ma.masked_array(np.zeros(cosmicMask.shape), cosmicMask, fill_value=1).filled(1).astype(np.uint8)

final_mask.add_mask(cosmicMask.astype(np.uint8), 4)

<ipython-input-17-6fba34c5c541>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cosmicMask, _ = mar.reduction.lacosmicx.lacosmicx(np.asarray(file_data[0].data, dtype=np.float32), final_mask.get_mask([1, 2, 64], binary=True).astype(np.uint8))


In [18]:
fits.PrimaryHDU(data=_).writeto("data/tests/return_fromlacosmicx.fits", overwrite=True)

In [19]:
fits.PrimaryHDU(data=cosmicMask).writeto("data/tests/cosmicmask.fits", overwrite=True)

In [ ]:

MarManager.INFO(f"applying cosmic mask to {file_name}")
im[0].data = apply_mask(im[0].data, cosmicMask, final_mask.get_mask([64], binary=True).astype(np.uint8))

In [32]:
final_mask.get_mask([1,2, 4, 8], binary=True, inverted=True, dtype='float32')

MaskArray([[0., 0., 0., ..., 0., 0., 0.],
           [0., 1., 0., ..., 0., 1., 0.],
           [1., 1., 1., ..., 1., 1., 1.],
           ...,
           [0., 1., 1., ..., 1., 1., 0.],
           [0., 0., 1., ..., 1., 0., 0.],
           [0., 0., 1., ..., 1., 0., 0.]], dtype=float32)

In [27]:
mar.env.marConf['config']['FILTER_SATELLITE']

True